In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tools
import schottky as sch
import fitutils as ft

In [2]:

# Data import

from data import temperature
from data import sample_HC
from data import err_sample_HC
from data import err_temperature

squared_temperature = temperature**2  # K**2
C_div_T = sample_HC/temperature  # mJ/K**2.mol
err_C_divT = err_sample_HC/temperature

In [3]:
# Constants

k = 1.380649e-23 # Boltzmann, J/K
delta = 2.9461005*k*2.4 
r = 8.31446261815324  # J/mol.K

### I. Schottky anomaly study

### II. Linear regression 

### III. Non-linear regression